In [6]:
import numpy as np
from sqlite3 import *
import pandas as pd
from math import *
import sympy as smp
import Cubic_Equation


class FluidProperties:
    """
    This helps You to find Fugacity.

    Pressure is in psi
    Temperature is in degree of Fahrenheit
    """

    def __init__(self, P, T):
        self.P = P
        self.T = T

    # name, Tc, Pc, W, https://github.com/CorySimon/PREOS/blob/master/preos.py

    def Composition(self, num):

        """
            Pass parameters desribing molecules
            :param num: Number of molecules in the mixture
            all Data entered are in oil field Unit
        """

        self.num = num

        # Create a database or connect to one

        conn = connect('fluid_properties.db')

        # Create a Cursor

        c = conn.cursor()

        # Create a Table

        c.execute(
            """create table IF NOT EXISTS fluid_properties (name text unique not null, Zi float null, Xi float null, 
            Yi float null, Tc float null, Pc float null, Mw float null, acenteric_factor float null , Gama float 
            null)""")

        # All Properties which will be taken

        properties = ['Zi', 'Xi', 'Yi', 'Tc', 'Pc', 'Mw', 'acenteric_factor', 'Gama']

        # Making a query set for properties in numpy for further calculations

        self.main_prop = np.zeros((self.num, len(properties)))
        self.comp_name = []

        # For loop for all components for which we take the properties

        text_list = []

        for i in range(self.num):
            self.name = str(input('What is the name of the component (say C1 or N2): '))
            self.comp_name.append(self.name)

            # Try and except for circumstances where a component is given which already exists in db
            # Try is ran only when a component is given that is quite new to db

            try:
                c.execute("""INSERT into fluid_properties (name) values (?)""", (self.name,))
                c.execute(f"""SELECT name FROM fluid_properties WHERE fluid_properties.name=(?)""", (self.name,))
                select = c.fetchall()
                for nam in select:
                    if nam[0] == self.name:
                        for j in range(len(properties)):
                            temp = input(f'please enter the {properties[j]}: ')
                            if temp == '':
                                change = f"""UPDATE fluid_properties SET {properties[j]}=NULL WHERE name='{self.name}'"""
                                self.main_prop[i, j] = np.nan
                            else:
                                change = f"""update fluid_properties set {properties[j]}={temp} WHERE name='{self.name}'"""
                                self.main_prop[i, j] = temp
                            c.execute(change)

            # Except is ran only when an already existed component is given

            except:
                if self.name == 'C7+':
                    for j in range(len(properties)):
                        temp = input(f'please enter the {properties[j]}: ')
                        if temp == '':
                            change = f"""UPDATE fluid_properties SET {properties[j]}=NULL WHERE name='{self.name}'"""
                            self.main_prop[i, j] = np.nan
                        else:
                            change = f"""update fluid_properties set {properties[j]}={temp} WHERE name='{self.name}'"""
                            self.main_prop[i, j] = temp
                        c.execute(change)

                # C7+ component properties always is new to db since they do not have a specified and determined
                # properties.

                if self.name != 'C7+':
                    c.execute("""UPDATE fluid_properties SET Zi=NULL WHERE name=(?)""", (self.name,))
                    c.execute("""UPDATE fluid_properties SET Xi=NULL WHERE name=(?)""", (self.name,))
                    c.execute("""UPDATE fluid_properties SET Yi=NULL WHERE name=(?)""", (self.name,))

                    # This for loop is for times when an empty data is given which means the value is unkown

                    for h in range(3):
                        temp = input(f'please enter the {properties[h]}: ')
                        if temp == '':
                            change = f"""UPDATE fluid_properties SET {properties[h]}=NULL WHERE name='{self.name}'"""
                            self.main_prop[i, h] = np.nan
                        else:
                            change = f"""update fluid_properties set {properties[h]}={temp} WHERE name='{self.name}'"""
                            self.main_prop[i, h] = temp
                        c.execute(change)

                c.execute("""select * from fluid_properties where name=(?)""", (self.name,))
                comp = c.fetchone()
                for data in range(len(comp) - 1):
                    self.main_prop[i, data] = comp[data + 1]

            # For printing the result

            text = f"""Molecule: {self.name}"""
            for j in range(len(properties)):
                text += f""", {properties[j]}: {self.main_prop[i, j]}"""

            text_list.append(text)
        self.Print = np.array(text_list)

        conn.commit()
        # Commit the changes

        conn.commit()
        conn.close()

        # Printing the result of the function

        dictionary = dict(zip(properties, list(self.main_prop.transpose())))
        update = dict(zip(['Component', ], [self.comp_name]))
        dictionary.update(update)
        self.df = pd.DataFrame(dictionary).set_index('Component')

        return self.df

    def EOS(self, p, t):

        self.p = p
        self.T = t
        self.df["ai"] = list(0.42748 * 10.731**2 * self.df["Tc"]**2.5 / (self.df["Pc"]))
        self.df["bi"] = list((0.08664 * 10.731 * self.df["Tc"]) / (self.df["Pc"]))

        self.df["Xi×bi"] = list(self.df["Xi"] * self.df["bi"])
        self.df["Yi×bi"] = list(self.df["Yi"] * self.df["bi"])

        am_g = ((self.df["Yi"])*(self.df["ai"]**0.5).sum(axis=0))**2
        bm_g = ((self.df["bi"])*(self.df["Yi"])).sum(axis=0)
        A_g = (am_g * self.p) / ((10.731**2) * (self.T**2.5))
        B_g = (bm_g * self.p) / (10.731 * self.T)

        am_l = (((self.df["Xi"]) * (self.df["ai"]**0.5)).sum(axis=0))**2
        bm_l = ((self.df["bi"]) * (self.df["Xi"])).sum(axis=0)
        A_l = (am_l * self.p) / (10.731**2 * (self.T**2.5))
        B_l = (bm_l * self.p) / (10.731 * self.T)

        df2 = pd.DataFrame({'Phase': ["Liquid", "Gas"], 'am': [am_l, am_g], 'bm': [bm_l, bm_g], 'A': [A_l, A_g], 'B': [B_l, B_g]})

        df2["First_coef"] = -(df2.iloc[1, 3]**2 + df2.iloc[1, 3] - df2.iloc[1, 2])
        df2["Second_coef"] = -(df2.iloc[1, 3]*df2.iloc[1, 2])
        print(df2)
        gas_Zfactor = Cubic_Equation.solve(1, -1, df2.iloc[1, 4], df2.iloc[1, 5])

        for i in gas_Zfactor:
            if i.real < 0 or type(i) == complex:
                continue
            if i > 0:
                gas_Zfactor = i

        liquid_Zfactor = Cubic_Equation.solve(1, -1, df2.iloc[0, 4], df2.iloc[0, 5])

        for i in liquid_Zfactor:
            if i.real < 0 or type(i) == complex:
                continue
            if i > 0:
                liquid_Zfactor = i

        df2["Z Factor"] = [liquid_Zfactor, gas_Zfactor]

        # fugacity p*exp(integral((Z-1)/p), 0, p)
        # integral(((Z-1)/p), 0, p) = Z - 1 - ln(Z - Bp) - (A**2/B)*ln(1 + (B*p/Z))

        df2["Gas_Fugacity"] = self.p * exp(gas_Zfactor - 1 - log(gas_Zfactor - B*self.p) - (A**2/B) * log(1 + (B*self.p/Z)))

        print(self.df)

        # @staticmethod

    def GasProperties(self):

        """
        This method gets the prerequisite data form __init__ method and calculates the properties of the gas.
        Note: All parameters are in Oil Field unit!
        If you do not have the value of each parameter pass zero(0).
        :return:
        """
        model = input('Proceeding "compositional" or "black oil" modeling: ')
        default = 0
        self.sour = input('if we have CO2 or H2S or N2 type "y" = ') or 'n'
        P = self.P
        T = self.T + 460
        self.Z = input('Z = ') or default
        self.Z = float(self.Z)

        if model == 'black oil':
            self.Gas_Mw = input('Mw = ') or default
            self.Gas_Mw = float(self.Gas_Mw)
            self.Gas_Gamma = input('Gama = ') or default
            self.Gas_Gamma = float(self.Gas_Gamma)
            self.Ppc = input('Ppc = ') or default
            self.Ppc = float(self.Ppc)
            self.Tpc = input('Tpc = ') or default
            self.Tpc = float(self.Tpc)
            self.Bg = input('Bg(ft3/scf) = ') or default
            self.Bg = float(self.Bg)
            self.Gas_density = input('ro(lbm/ft3) = ') or default
            self.Gas_density = float(ro)
            self.GPG = input('GPG(psi/ft) = ') or default
            self.GPG = float(self.GPG)
            self.cg = input('Cg(1/psi) = ') or default
            self.cg = float(self.cg)
            self.Gas_viscosity = input('viscosity(CP) = ') or default
            self.Gas_viscosity = float(self.Gas_viscosity)

            if self.Tpc == 0:
                self.Tpc = 169.2 + 349.5 * self.Gas_Gamma - 74 * self.Gas_Gamma ** 2
                print(f'Tpc = {self.Tpc}')

            if self.Ppc == 0:
                self.Ppc = 756.8 - 131 * self.Gas_Gamma - 3.6 * self.Gas_Gamma ** 2
                print(f'Ppc = {self.Ppc}')

            if self.sour == 'y':
                c = input('yCO2 = ')
                s = input('yH2S = ')
                n = input('yN2 = ')
                self.Tpc = self.Tpc - 80 * c + 130 * s - 250 * n
                self.Ppc = self.Ppc + 440 * c + 600 * s - 170 * n

            self.Ppr = self.P / self.Ppc
            self.Tpr = self.T / self.Tpc

            print(f'Ppr = {self.Ppr}')
            print(f'Ppr = {self.Ppr}')

            if self.Gas_Mw == 0:
                self.Gas_Mw = 28.97 * self.Gas_Gamma
                print(f'Mw = {self.Gas_Mw}')

            if self.Gas_Gamma == 0:
                self.Gas_Gamma = self.Gas_Mw / 28.96
                print(f'Gama = {self.Gas_Gamma}')

            if self.Z == 0:
                self.Z = 1 - ((3.53 * self.Ppr) / (10 ** (0.9813 * self.Tpr))) + ((0.274 * self.Ppr ** 2) / (10 ** (0.8157 * self.Tpr)))
                print(f'Z = {self.Z}')

            if self.Gas_density == 0:
                self.Gas_density = (self.P * self.Gas_Mw) / (self.Z * self.T * 10.732)
                print(f'ro = {self.Gas_density}')

            if self.GPG == 0:
                self.GPG = self.Gas_density / 144
                print(f'GPG = {self.GPG}')

            if self.Bg == 0:
                self.Bg = 0.0283 * self.Z * self.T / self.P
                print(f'Bg = {self.Bg}')

            if self.Gas_viscosity == 0:
                A = ((9.379 + 0.01607 * self.Gas_Mw) * self.T ** 1.5) / (209.2 + 19.26 * self.Gas_Mw + self.T)
                B = 3.448 + (986.4 / self.T) + 0.01009 * self.Gas_Mw
                C = 2.447 - 0.2224 * B
                self.Gas_viscosity = (10 ** -4) * A * exp(B * (self.Gas_density / 62.4) ** C)
                print(f'mio = {self.Gas_viscosity}')
        elif model == 'compositional':
            Composition = self.Composition(int(input('please enter the number of component: ')) or self.num)
#             print(Composition)
            self.Gas_Mw = sum(Composition['Yi'] * Composition['Mw'])
            self.Gas_Gamma = self.Gas_Mw / 28.96
#             print(f'Mw = {self.Gas_Mw}')

            if self.P < 3000 or self.Gas_Gamma < 0.75:
                self.Tpc = sum(Composition['Yi'] * Composition['Tc'])
            else:
                print('hello')
                print(Composition.reset_index)
                # self.Tpc = sum(Composition[''])

            #print(f'Tpc = {self.Tpc}')



a = FluidProperties(3001, 135)
# a.Composition(2)
# a.EOS(2000, 620)
b = a.GasProperties()

Proceeding "compositional" or "black oil" modeling: compositional
if we have CO2 or H2S or N2 type "y" = 
Z = 
please enter the number of component: 2
What is the name of the component (say C1 or N2): C1
please enter the Zi: 6
please enter the Xi: 6
please enter the Yi: 6
What is the name of the component (say C1 or N2): C7+
please enter the Zi: 7
please enter the Xi: 7
please enter the Yi: 7
please enter the Tc: 7
please enter the Pc: 7
please enter the Mw: 1000
please enter the acenteric_factor: 5
please enter the Gama: 6
hello
<bound method DataFrame.reset_index of             Zi   Xi   Yi     Tc     Pc      Mw  acenteric_factor  Gama
Component                                                             
C1         6.0  6.0  6.0  100.0  100.0    16.0              0.03   0.3
C7+        7.0  7.0  7.0    7.0    7.0  1000.0              5.00   6.0>
